# Understanding the Graph and Session in TF

TensorFlow is a powerful open source software library for numerical computation, particularly well suited and fine-tuned for large-scale Machine Learning. 

Its basic principle is simple: 
- Define a graph of computations to perform in Python;
- TensorFlow takes that graph and runs it efficiently using optimized C++ code.

Most importantly, it is possible to break up the graph into several chunks and run them in parallel across multiple CPUs or GPUs.

# Creating the First Graph and Running It in a Session

In [1]:
import tensorflow as tf

x = tf.Variable(3, name = 'x')
y = tf.Variable(3, name = 'y')
f = x*x*y + y + 2

/Users/caihaocui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The most important thing to understand is that this code does not actually perform any computation, even though it looks like it does (especially the last line). It just creates a computation graph. 

In fact, even the variables are **not initialized** yet. 

To evaluate this graph, you need to open a TensorFlow session and use it to initialize the variables and evaluate `f`.

The following code creates a session, initializes the variables, and evaluates, and f then closes the session (which frees up resources):

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

32


Having to repeat sess.run() all the time is a bit cumbersome, but fortunately there is a better way:

In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

32


Inside the with block, the session is set as the default session. Calling `x.initializer.run()` is equivalent to calling `tf.get_default_session().run(x.initializer)`, and similarly `f.eval()` is equivalent to calling `tf.get_default_session().run(f)`. This makes the code easier to read. 

Moreover, the session is automatically closed at the end of the block.

Instead of manually running the initializer for every single variable, you can use the `global_variables_initializer()` function. Note that it does not actually perform the initialization immediately, but rather creates a node in the graph that will initialize all variables when it is run:

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)

32


Inside Jupyter or within a Python shell you may prefer to create an InteractiveSes sion. The only difference from a regular Session is that when an InteractiveSes sion is created it automatically sets itself as the default session, so you don’t need a with block (but you do need to close the session manually when you are done with it):

In [7]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

32


A TensorFlow program is typically split into two parts: 
- **construction phase** - build a computation graph;
- **execution phase**    - run the graph (do the computation). 

The construction phase typically builds a computation graph representing the ML model and the computations required to train it. The execution phase generally runs a loop that evaluates a training step repeatedly (for example, one step per mini-batch), gradually improving the model parameters.

# Managing Graphs

Any node you create is automatically added to the default graph:

In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In most cases this is fine, but sometimes you may want to manage multiple independent graphs. You can do this by creating a new Graph and temporarily making it the default graph inside a with block, like so:

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [11]:
x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph()

False

In Jupyter (or in a Python shell), it is common to run the same commands more than once while you are experimenting. As a result, you may end up with a default graph containing many duplicate nodes. One solution is to restart the Jupyter kernel (or the Python shell), but a more convenient solution is to just reset the default graph by running `tf.reset_default_graph()`.

In [15]:
tf.reset_default_graph()

# Lifecycle of a Node Value

When you evaluate a node, TensorFlow automatically determines the set of nodes that it depends on and it evaluates these nodes first. For example, consider the follow‐ ing code:

In [19]:
w = tf.constant(9)
x = w + 1
y = x + 1
z = x + 2
with tf.Session() as sess:
    print(y.eval()) # 11
    print(z.eval()) # 12

11
12


First, this code defines a very simple graph. Then it starts a session and runs the graph to evaluate `y`: TensorFlow automatically detects that `y` depends on `w`, which depends on `x`, so it first evaluates `w`, then `x`, then `y`, and returns the value of `y`. Finally, the code runs the graph to evaluate `z`. 

Once again, TensorFlow detects that it must first evaluate `w` and `x`. It is important to note that it will not reuse the result of the previous evaluation of `w` and `x`. In short, the preceding code evaluates `w` and `x` twice.

**All node values are dropped between graph runs, except variable values, which are maintained by the session across graph runs. A variable starts its life when its initializer is run, and it ends when the session is closed. **

If you want to evaluate `y` and `z` efficiently, without evaluating `w` and `x` twice as in the previous code, you must ask TensorFlow to evaluate both `y` and `z` in just one graph run, as shown in the following code:

In [20]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

11
12


In single-process TensorFlow, multiple sessions do not share any state, even if they reuse the same graph (each session would have its own copy of every variable).

In distributed TensorFlow, variable state is stored on the servers, not in the sessions, so multiple sessions can share the same variables.

**The End**